In [1]:
1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1695658432399_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [2]:

import logging
import sys
import time
import traceback
import boto3
import pandas as pd
from common_utils import common_functions as cf
from common_utils import exception_helpers as eh
import json
from common_utils import db_helpers
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
args = {'aws_read_shared_vpc': 'arn:aws:iam::336913029742:role/admin_access_for_{}_from_shared', 'EMAIL_RESPONSE_SLA_PATH': 'common/Email_Response_SLA.csv', 'feed_attribute_object': 'common/feed_attr.cfg', 'feed_metadata_object': 'common/feed_metadata.cfg', 'uow_dependency_object': 'common/uow_dependency.cfg', 'uow_error_object': 'common/uow_error.cfg', 'MasterSecurityGroupName': 'AirflowEMRMasterSG', 'SlaveSecurityGroupName': 'AirflowEMRSlaveSG', 'SubnetKeyName': 'data-partner01-{}-private-app-2a', 'DB_CLUSTER_IDENTIFIER': 'auroraclusteroperational', 'instance_profile_role': 'ec2_run_terraform', 'service_role': 'EC_EMR_DEFAULTBBI', 'auto_scaling_role': 'EC_EMR_DEFAULTBBI', 'EMR_EC_PEM_FILE': 'EC2_BBI_DEV', 'ServiceSecurityGroupName': 'AirflowServiceSecurityGroup', 'avant_source_type': 'Regular', 'avant_reproc_source_type': 'Reproc', 'loaded_by': 'avantadmin', 'config_file_bucket': 'avantdlds-prod-common-bucket', 'config_file_name': 'common/feed_metadata.cfg', 'parameter_config_file': 'common/parameters.cfg', 'region': 'us-east-2', 'feed_attribute_config_file': 'common/feed_attr.cfg', 'mailbox_csv_path': '/common/Mailboxes.csv', 'cluster_id': 'cluster_id', 'spark_application_type': 'pyspark', 'aws_connection_id': 'aws_default', 'OPS_TABLE_S3_BUCKET_TABLE': 'S3_BUCKET_TABLE', 'OPS_TABLE_BATCH_RUN_STATS': 'BATCH_RUN_STATS', 'OPS_TABLE_BATCH_REPORTS': 'BATCH_REPORTS', 'transform_bucket': 'avantdlds-partner01-prod-transform-bucket', 'common_bucket': 'avantdlds-partner01-prod-common-bucket', 'exception_bucket': 'avantdlds-partner01-prod-exception-bucket', 'log_bucket': 'avantdlds-partner01-prod-log-bucket', 'raw_bucket': 'avantdlds-partner01-prod-raw-bucket', 'stage_bucket': 'avantdlds-partner01-prod-stage-bucket', 'transient_bucket': 'avantdlds-partner01-prod-transient-bucket', 'drop_bucket': 'avant-partner01-landing', 'outgoing_bucket': 'avantdlds-partner01-prod-outgoing-bucket', 'dremio_security_table_name': 'dremio_security', 'dedup_loan_by_installment': 'loan_by_installment/loan_by_installment_002/dedup_loan_by_installment/', 'partner01_source': 'avantdlds-partner01-prod', 'ref_data_object': 'common/reference_data.cfg', 'decrypt_script_path': 'dag/parallel_decrypt.py', 'aps_dedup_active_installment': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_active_installment', 'apu_dedup_active_installment': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_active_installment', 'dremio_security_redesign': '"Security".dremio_security_redesign', 'aps_dedup_balances': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_balances', 'aps_dedup_interest_rate_schedule': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_interest_rate', 'aps_dedup_loan_debt_sale': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_loan_debt_sale', 'aps_dedup_loan_financial_owner': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_loan_financial_owner', 'aps_dedup_loan_payment_plan': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_loan_payment_plan', 'aps_dedup_loan_servicing_settlement': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_loan_servicing_settlement', 'aps_dedup_treasury_charge_off': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_treasury_charge_off', 'apu_dedup_balances': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_balances', 'apu_dedup_interest_rate_schedule': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_interest_rate', 'apu_dedup_loan_debt_sale': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_loan_debt_sale', 'apu_dedup_loan_financial_owner': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_loan_financial_owner', 'apu_dedup_loan_payment_plan': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_loan_payment_plan', 'apu_dedup_loan_servicing_settlement': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_loan_servicing_settlement', 'apu_dedup_treasury_charge_off': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_treasury_charge_off', 'aps_dedup_operational_charge_off': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_operational_charge_off', 'apu_dedup_operational_charge_off': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_operational_charge_off', 'aps_dedup_original_installment': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/dedup_original_installment', 'apu_dedup_original_installment': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/dedup_original_installment', 'loan_by_day': 'loan_by_day/loan_by_day_003/dedup_loan_by_day/', 'loan_payment_plan_v3_union': 'account_portfolios_union/account_portfolios_union_001/loan_payment_plan/loan_payment_plan_union', 'dedup_loan_by_day': 'loan_by_day/loan_by_day_003/dedup_loan_by_day', 'loan_financial_owner_v3_union': 'account_portfolios_union/account_portfolios_union_001/loan_financial_owner/loan_financial_owner_union', 'loan_by_day_v3': 'loan_by_day/loan_by_day_003/dedup_loan_by_day/', 'raw_loan_by_day': 'loan_by_day/loan_by_day_003/loan_by_day/*/*/*/*/*/', 'dedup_loan_balance_by_day': 'loan_balance_by_day/loan_balance_by_day_001/loan_balance_by_day/', 'original_installment_union': 'account_portfolios_union/account_portfolios_union_001/original_installment/original_installment_union', 'cancelled_loans_union': 'account_portfolios_union/account_portfolios_union_001/cancelled_loans/cancelled_loans_union', 'dedup_loan_transcation_v3': 'loan_transaction/loan_transaction_003/dedup_loan_transaction/', 'fd_cc_lv_account_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_001/dedup_account', 'fd_cc_lv_account_delinquency_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_002/dedup_account_delinquency', 'fd_cc_lv_account_delinquency_items_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_003/dedup_account_delinquency_items', 'fd_cc_lv_current_monetary_activity_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_004/dedup_current_monetary_activity', 'fd_cc_lv_fraud_falcon_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_005/dedup_fraud_falcon', 'fd_cc_lv_historical_cardholder_data_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_006/dedup_historical_cardholder_data', 'interest_rate_union': 'account_portfolios_union/account_portfolios_union_001/interest_rate/interest_rate_union', 'treasury_charge_off_union': 'account_portfolios_union/account_portfolios_union_001/treasury_charge_off/treasury_charge_off_union', 'loan_financial_owner_union': 'account_portfolios_union/account_portfolios_union_001/loan_financial_owner/loan_financial_owner_union', 'loan_debt_sale_union': 'account_portfolios_union/account_portfolios_union_001/loan_debt_sale/loan_debt_sale_union', 'loan_servicing_settlement_union': 'account_portfolios_union/account_portfolios_union_001/loan_servicing_settlement/loan_servicing_settlement_union', 'dedup_loan_transaction': 'loan_transaction/loan_transaction_003/dedup_loan_transaction', 'fd_cc_lv_monetary_detail_charge_off_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_007/dedup_monetary_detail_charge_off', 'fd_cc_lv_monetary_detail_cross_cycle_adjustment_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_008/dedup_monetary_detail_cross_cycle_adjustment', 'fd_cc_lv_monetary_detail_current_cycle_adjustment_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_009/dedup_monetary_detail_current_cycle_adjustment', 'fd_cc_lv_monetary_detail_transaction_data_lake_dedup_file': 'fd_cc_lv/fd_cc_lv_010/dedup_monetary_detail_transaction', 'tco_reference_file_path': 'tco_reference/tco_ref_update.csv', 'aps_dedup_cancelled_loans': 'account_portfolios_refi/account_portfolios_refi_001/account_portfolios_refi/cancelled_loans/append_cancelled_loans', 'apu_dedup_cancelled_loans': 'account_portfolios_uil/account_portfolios_uil_001/account_portfolios_uil/cancelled_loans/append_cancelled_loans', 'parquet_block_size': '512', 'uil_raw_dedup_path': 'account_portfolios_uil/account_portfolios_uil_001/dedup_account_portfolios_uil/', 'refi_raw_dedup_path': 'account_portfolios_refi/account_portfolios_refi_001/dedup_account_portfolios_refi/', 'balances_union': 'loan_balance_by_day/loan_balance_by_day_001/balances_union/', 'dremio_security_table_path': 'Security.dremio_security_redesign', 'prospect_match_campaign_segments_filename': 'campaign_segments.pq', 'prospect_match_campaign_segments_transient_data_path': 'direct_mail/send_dictonary/campaign_segments', 'prospect_match_source_map_filename': 'prospect_match_source_mapping.pq', 'prospect_match_source_map_transient_data_path': 'source_attribution_v2/source_attribution_001_v2/prospect_match/source_mapping/2023/09/23', 'prospect_match_data_path': 'source_attribution_v2/source_attribution_001_v2/prospect_match', 'prospect_match_raw_data_path': 'source_attribution_v2/source_attribution_001_v2/prospect_match/2023/09/23', 'status_dict': "{'step_1': 'Landed', 'step_2': 'Ingested', 'step_3': 'Registered', 'step_4': 'Staged', 'step_5': 'Transformed', 'step_6': 'Expired', 'step_7': 'Rejected', 'step_8':'raw_preprocessed'}", 'prospect_match_campaign_segments_transient_filename': 'campaign_segments_265828_1_20230923.pq', 'prospect_match_source_map_transient_filename': 'source_mapping_265828_1_20230923.pq', 'prospect_match_loan_app_raw_vds_path': 'avant.{}.dl.source_attribution_v2.source_attribution_loan_customer_application', 'prospect_match_loan_app_raw_pds_path': 'avantdlds-partner01-prod.avantdlds-partner01-{}-raw-bucket.{}.source_attribution_v2.source_attribution_001_v2.source_attribution_loan_app', 'prospect_match_cc_app_raw_pds_path': 'avantdlds-partner01-prod.avantdlds-partner01-{}-raw-bucket.{}.source_attribution_v2.source_attribution_001_v2.source_attribution_cc_app', 'prospect_match_cc_app_raw_vds_path': 'avant.{}.dl.source_attribution_v2.source_attribution_cc_customer_application', 'prospect_match_loan_raw_vds_path': 'avant.{}.dl.source_attribution_v2.source_attribution_loan', 'prospect_match_loan_raw_pds_path': 'avantdlds-partner01-prod.avantdlds-partner01-{}-raw-bucket.{}.source_attribution_v2.source_attribution_001_v2.source_attribution_loan', 'prospect_match_cc_raw_pds_path': 'avantdlds-partner01-prod.avantdlds-partner01-{}-raw-bucket.{}.source_attribution_v2.source_attribution_001_v2.source_attribution_cc', 'prospect_match_cc_raw_vds_path': 'avant.{}.dl.source_attribution_v2.source_attribution_cc_account', 'prospect_match_loan_app_path_uil': 'loan_customer_application/loan_customer_application_013/loan_customer_application_ver_join', 'prospect_match_loan_app_path_refi': 'loan_customer_application/loan_customer_application_033/loan_customer_application_ver_join', 'prospect_match_cc_path': 'cc_account/cc_account_001/cc_account', 'prospect_match_loan_path': 'loan/loan_001/loan', 'prospect_match_tealium_path': 'tealium/tealium_001/tealium_event_dedup', 'prospect_match_mailed_selected_path': 'avantcamp-partner01-{}-transform-bucket/avant/direct_mail/first_mail/creative_code_allocated_and_deduped_history/', 'prospect_match_cc_app_path': 'credit_card_customer_application/credit_card_customer_application_004/credit_card_customer_application_final', 'leads_source_path': 'avant/marketing_pre_app/leads_001/dedup_lead/*/*/*/*/*', 'prospect_match_mailed_path': 'avant/direct_mail/mailed_deduped_source_attribution_data/direct_mail_mailed_deduped_history.pq', 'operational_charge_off_v3_union': 'account_portfolios_union/account_portfolios_union_001/operational_charge_off/operational_charge_off_union', 'VAULT_URL': 'https://vault.shared.partner01.data.avant.com', 'vault_encrypt_path': 'app/landing_zone_encryption/amount/prod', 'ABC_VAULT_PATH': 'app/abc/prod', 'secret_path': 'app/abc/prod', 'DREMIO_USERNAME': 'XXXXXXXXXX', 'DREMIO_PASSWORD': 'XXXXXXXXXX', 'file_id': 'source_attribution_001_v2', 's3_bucket_process_start_time': '2023-09-23 21:54:06', 'ec2_role': 'airflow-role', 'role_name': 'avant-dlds-airflow-prod-ecs-task-role', 'emr_role_name': 'ec2_run_terraform', 'metadata_url_base': 'http://169.254.169.254', 'batch_id': '265828', 'run_id': '1', 'abc_attribute_list': [('send_number', '1', 'send_number', 'int'), ('campaign_name', '2', 'campaign_name', 'varchar'), ('coupon_code', '3', 'coupon_code', 'varchar'), ('creative_group_id', '4', 'creative_group_id', 'int'), ('drop_date', '14', 'drop_date', 'date'), ('selection_group_id', '5', 'selection_group_id', 'int'), ('policy_version', '6', 'policy_version', 'varchar'), ('default_model_version', '7', 'default_model_version', 'varchar'), ('default_model_2_version', '8', 'default_model_2_version', 'varchar'), ('bk_model_version', '9', 'bk_model_version', 'varchar'), ('product_type', '10', 'product_type', 'varchar'), ('selection_size', '11', 'selection_size', 'int'), ('mailed_size', '12', 'mailed_size', 'int'), ('est_resp_rate', '13', 'est_resp_rate', 'float'), ('start_date', '15', 'start_date', 'date'), ('end_date', '16', 'end_date', 'date'), ('remail', '17', 'remail', 'boolean'), ('per_unit_cost', '18', 'per_unit_cost', 'float'), ('is_bau_selection', '19', 'is_bau_selection', 'boolean'), ('is_creative_test', '20', 'is_creative_test', 'boolean'), ('credit_line', '21', 'credit_line', 'varchar'), ('id', '1', 'id', 'int'), ('priority', '2', 'priority', 'int'), ('channel', '3', 'channel', 'varchar'), ('name', '4', 'name', 'varchar'), ('key', '14', 'key', 'varchar'), ('value', '5', 'value', 'varchar'), ('special_op', '6', 'special_op', 'varchar'), ('source_window', '7', 'source_window', 'varchar'), ('start_dt', '8', 'start_dt', 'date'), ('end_dt', '9', 'end_dt', 'date')], 'combined_attribute_list': [], 'partner_id': 'partner01', 'env': 'prod', 'sub_partner': 'avant', 'application': 'source_attribution_v2', 'business_date': '20230923'}
args['job_id'] = 6291613
args['program_name'] = 'campaign_tables_ingest_to_transient.py'




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
process_start_time = cf.get_cur_time()
SPARK = SparkSession.builder.appName(args['file_id']).getOrCreate()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:

campaign_segments_raw_filename = args['prospect_match_campaign_segments_transient_filename']
campaign_segments_raw_data_path = args['prospect_match_campaign_segments_transient_data_path']
source_map_raw_filename = args['prospect_match_source_map_transient_filename']
source_map_raw_data_path = args['prospect_match_source_map_transient_data_path']
file_id = args['file_id']
sub_partner = args['sub_partner']
transient_bucket = args['transient_bucket']
transient_target_campaign_segments = 's3://' + transient_bucket + '/' + sub_partner + '/' + campaign_segments_raw_data_path + '/' + campaign_segments_raw_filename
transient_target_source_map = 's3://' + transient_bucket + '/' + sub_partner + '/' + source_map_raw_data_path + '/' + source_map_raw_filename


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
    current_epoch_time = int(time.time())
    current_time = cf.get_cur_time()

    table_keys = {'campaign_segments': ['send_number', 'campaign_name', 'coupon_code', 'creative_group_id', 'drop_date',
                                        'selection_group_id', 'policy_version', 'default_model_version',
                                        'default_model_2_version', 'bk_model_version',
                                        'product_type', 'selection_size', 'mailed_size', 'est_resp_rate', 'start_date',
                                        'end_date', 'remail', 'per_unit_cost',
                                        'is_bau_selection', 'is_creative_test', 'credit_line'],
                  'source_mapping': ['id', 'priority', 'channel', 'name', 'key', 'value',
                                     'special_op', 'source_window', 'start_dt', 'end_dt']}



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
    for key, value in table_keys.items():
        print(key,value)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

campaign_segments ['send_number', 'campaign_name', 'coupon_code', 'creative_group_id', 'drop_date', 'selection_group_id', 'policy_version', 'default_model_version', 'default_model_2_version', 'bk_model_version', 'product_type', 'selection_size', 'mailed_size', 'est_resp_rate', 'start_date', 'end_date', 'remail', 'per_unit_cost', 'is_bau_selection', 'is_creative_test', 'credit_line']
source_mapping ['id', 'priority', 'channel', 'name', 'key', 'value', 'special_op', 'source_window', 'start_dt', 'end_dt']

In [10]:
key = 'campaign_segments'
value = ['send_number', 'campaign_name', 'coupon_code', 'creative_group_id', 'drop_date', 'selection_group_id', 'policy_version', 'default_model_version', 'default_model_2_version', 'bk_model_version', 'product_type', 'selection_size', 'mailed_size', 'est_resp_rate', 'start_date', 'end_date', 'remail', 'per_unit_cost', 'is_bau_selection', 'is_creative_test', 'credit_line']
key, value

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('campaign_segments', ['send_number', 'campaign_name', 'coupon_code', 'creative_group_id', 'drop_date', 'selection_group_id', 'policy_version', 'default_model_version', 'default_model_2_version', 'bk_model_version', 'product_type', 'selection_size', 'mailed_size', 'est_resp_rate', 'start_date', 'end_date', 'remail', 'per_unit_cost', 'is_bau_selection', 'is_creative_test', 'credit_line'])

In [27]:
key = 'source_mapping'
value = ['id', 'priority', 'channel', 'name', 'key', 'value', 'special_op', 'source_window', 'start_dt', 'end_dt']
key, value

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('source_mapping', ['id', 'priority', 'channel', 'name', 'key', 'value', 'special_op', 'source_window', 'start_dt', 'end_dt'])

In [28]:
        # read records from campaign_segments ABC
        query = f"""
        select * from public.{key}
        """
        print(query)

        records = db_helpers.get_records(args, file_id, query)
        col_names = [i[0] for i in args['abc_attribute_list'] if i[0] in value]

        # create spark dataframe from pandas
        pd_df = pd.DataFrame(records, columns=col_names)
        spark_df = SPARK.createDataFrame(pd_df)

        type_map = {
            'int': 'long',
            'varchar': 'string',
            'float': 'double',
            'date': 'date',
            'boolean': 'boolean'
        }


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
        print(query)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


select * from public.source_mapping

In [30]:
args['abc_attribute_list']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('send_number', '1', 'send_number', 'int'), ('campaign_name', '2', 'campaign_name', 'varchar'), ('coupon_code', '3', 'coupon_code', 'varchar'), ('creative_group_id', '4', 'creative_group_id', 'int'), ('drop_date', '14', 'drop_date', 'date'), ('selection_group_id', '5', 'selection_group_id', 'int'), ('policy_version', '6', 'policy_version', 'varchar'), ('default_model_version', '7', 'default_model_version', 'varchar'), ('default_model_2_version', '8', 'default_model_2_version', 'varchar'), ('bk_model_version', '9', 'bk_model_version', 'varchar'), ('product_type', '10', 'product_type', 'varchar'), ('selection_size', '11', 'selection_size', 'int'), ('mailed_size', '12', 'mailed_size', 'int'), ('est_resp_rate', '13', 'est_resp_rate', 'float'), ('start_date', '15', 'start_date', 'date'), ('end_date', '16', 'end_date', 'date'), ('remail', '17', 'remail', 'boolean'), ('per_unit_cost', '18', 'per_unit_cost', 'float'), ('is_bau_selection', '19', 'is_bau_selection', 'boolean'), ('is_creative_te

In [31]:
        # update typing and handle weird cases of nulls, NaNs, 0s
        for i in args['abc_attribute_list']:
            if i[0] in col_names:
                name_in = i[2]
                name_out = i[0]
                type_out = i[3]
                # temp columns with fixed typing
                spark_df = spark_df.withColumn(
                    name_out + '_temp',
                    spark_df[name_in].cast(type_map[type_out])
                )
                # remove original columns, and rename temps
                spark_df = spark_df.drop(name_in).withColumnRenamed(name_out + '_temp', name_out)
                # replace NaN with null - not sure why this happens, fixing here
                if type_out in ['int', 'float']:
                    spark_df = spark_df.withColumn(
                        name_out,
                        F.when(F.isnan(F.col(name_out)), None).otherwise(F.col(name_out))
                    )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
        # selection size 0 to null, not sure what makes this happen either
        if (key == 'campaign_segments'):
            spark_df = spark_df.withColumn(
                'selection_size',
                F.when(
                    F.col('selection_size') == F.lit(0),
                    None
                ).otherwise(F.col('selection_size'))
            )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
spark_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+------------+-------------------+------+--------------------+----------+-------------+--------+------+
| id|priority|     channel|               name|   key|               value|special_op|source_window|start_dt|end_dt|
+---+--------+------------+-------------------+------+--------------------+----------+-------------+--------+------+
|  2|       1|Partner-Comp|                ahl|source|                 ahl|      null|         null|    null|  null|
|  3|       1|Partner-Comp|           ahl-post|source|            ahl_post|      null|           30|    null|  null|
|  4|       1|Partner-Comp|              amone|source|               amone|      null|           30|    null|  null|
|  5|       1|Partner-Comp|         amone-post|source|      am_one_prefill|      null|           30|    null|  null|
|  6|       1|Partner-Comp|                ams|source|                 ams|      null|            3|    null|  null|
|  7|       1|Partner-Comp|      bankrate-post|source|          

In [34]:
        # get the creation time of the file in common bucket
        res = boto3.resource('s3')
        common_bucket = args['common_bucket']
        csv_key = f'source_attribution/{key}.csv'
        for i in res.Bucket(common_bucket).objects.filter(Prefix=csv_key):
            source_file_time = i.last_modified


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
        # add auditing columns
        spark_df = spark_df.withColumn(
            'etl_ingestion_time',
            F.lit(cf.get_cur_time()).cast('timestamp')
        ).withColumn(
            'source_file_creation_time',
            F.lit(F.lit(source_file_time)).cast('timestamp')
        )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
        record_count = spark_df.count()
        record_count


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

225069

In [37]:
        if key == 'campaign_segments':
            transient_target = transient_target_campaign_segments
        else:
            transient_target = transient_target_source_map
        spark_df.write.mode('overwrite').parquet(transient_target)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
        # Updating ABC database On success
        msg = f'Successfully saved data to transient bucket for source_attribution for {key}'
        status = 'Success'


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:

        db_helpers.update_on_success(
            args=args,
            process_start_time=process_start_time,
            status=status,
            source_filename='API',
            source_record_count=record_count,
            target_filename=transient_target,
            target_record_count=record_count,
            message=msg
        )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No active exception to reraise
Expected to work only for dev
Line to debug starts: http://169.254.169.254/latest/meta-data/iam/security-credentials/ec2_run_terraform
===========status code = 200==========
Already credentials exists
authentication was successful using credentials
Vault client authenticated
using approach 1
No active exception to reraise
Expected to work only for dev
Line to debug starts: http://169.254.169.254/latest/meta-data/iam/security-credentials/ec2_run_terraform
===========status code = 200==========
Already credentials exists
authentication was successful using credentials
Vault client authenticated
using approach 1
No active exception to reraise
Expected to work only for dev
Line to debug starts: http://169.254.169.254/latest/meta-data/iam/security-credentials/ec2_run_terraform
===========status code = 200==========
Already credentials exists
authentication was successful using credentials
Vault client authenticated
using approach 1

In [ ]:
    for key, value in table_keys.items():
        # read records from campaign_segments ABC
        query = f"""
        select * from public.{key}
        """

        records = db_helpers.get_records(args, file_id, query)
        col_names = [i[0] for i in args['abc_attribute_list'] if i[0] in value]

        # create spark dataframe from pandas
        pd_df = pd.DataFrame(records, columns=col_names)
        spark_df = SPARK.createDataFrame(pd_df)

        type_map = {
            'int': 'long',
            'varchar': 'string',
            'float': 'double',
            'date': 'date',
            'boolean': 'boolean'
        }

        # update typing and handle weird cases of nulls, NaNs, 0s
        for i in args['abc_attribute_list']:
            if i[0] in col_names:
                name_in = i[2]
                name_out = i[0]
                type_out = i[3]
                # temp columns with fixed typing
                spark_df = spark_df.withColumn(
                    name_out + '_temp',
                    spark_df[name_in].cast(type_map[type_out])
                )
                # remove original columns, and rename temps
                spark_df = spark_df.drop(name_in).withColumnRenamed(name_out + '_temp', name_out)
                # replace NaN with null - not sure why this happens, fixing here
                if type_out in ['int', 'float']:
                    spark_df = spark_df.withColumn(
                        name_out,
                        F.when(F.isnan(F.col(name_out)), None).otherwise(F.col(name_out))
                    )

        # selection size 0 to null, not sure what makes this happen either
        if (key == 'campaign_segments'):
            spark_df = spark_df.withColumn(
                'selection_size',
                F.when(
                    F.col('selection_size') == F.lit(0),
                    None
                ).otherwise(F.col('selection_size'))
            )

        # get the creation time of the file in common bucket
        res = boto3.resource('s3')
        common_bucket = args['common_bucket']
        csv_key = f'source_attribution/{key}.csv'
        for i in res.Bucket(common_bucket).objects.filter(Prefix=csv_key):
            source_file_time = i.last_modified

        # add auditing columns
        spark_df = spark_df.withColumn(
            'etl_ingestion_time',
            F.lit(cf.get_cur_time()).cast('timestamp')
        ).withColumn(
            'source_file_creation_time',
            F.lit(F.lit(source_file_time)).cast('timestamp')
        )

        record_count = spark_df.count()

        if key == 'campaign_segments':
            transient_target = transient_target_campaign_segments
        else:
            transient_target = transient_target_source_map
        spark_df.write.mode('overwrite').parquet(transient_target)

        # Updating ABC database On success
        msg = f'Successfully saved data to transient bucket for source_attribution for {key}'
        status = 'Success'

        db_helpers.update_on_success(
            args=args,
            process_start_time=process_start_time,
            status=status,
            source_filename='API',
            source_record_count=record_count,
            target_filename=transient_target,
            target_record_count=record_count,
            message=msg
        )


In [ ]:




try:

except Exception:
    logging.exception('exception block')
    exc_type, exc_value, exc_traceback = sys.exc_info()
    syslog_struct = "{}".format(str(traceback.format_exception(
        exc_type,
        exc_value, exc_traceback)))
    error_id = 'GEN_APP_001'
    syslog_struct = syslog_struct.replace("'", '"')
    # LOGGING TO THE ABC DATA BASE ON FAILURE
    db_helpers.update_on_failure(args=args,
                                 process_start_time=process_start_time,
                                 error_id=error_id,
                                 error_summary=syslog_struct,
                                 skip_down_stream_update=True)
    tb = json.dumps(traceback.format_tb(exc_traceback))
    raise eh.SparkSessionError(str(exc_type), str(exc_value), tb)
